In [1]:
import json
import numpy as np
import seismic
from seismic import SeismicDatasetLV, SeismicIndexLV
from tqdm import tqdm

In [3]:

document_path = ""



dataset = SeismicDatasetLV()
string_type  = seismic.get_seismic_string()

with open(document_path, "r") as file:
    for line in tqdm(file):
        line_data = json.loads(line.strip())
        vs = np.array([v for v in line_data["vector"].values()], dtype=np.float32)
        ks = np.array([k for k in line_data["vector"].keys()], dtype=string_type)
        dataset.add_document(str(line_data["id"]), ks, vs)
        

171332it [01:34, 1808.33it/s]


In [4]:
index = SeismicIndexLV.build_from_dataset(dataset)


Building the index...
Configuration { pruning: GlobalThreshold { n_postings: 3500, max_fraction: 1.5 }, blocking: RandomKmeans { centroid_fraction: 0.1, min_cluster_size: 2, clustering_algorithm: RandomKmeansInvertedIndexApprox { doc_cut: 15 } }, summarization: EnergyPreserving { summary_energy: 0.4 }, knn: KnnConfiguration { nknn: 0, knn_path: None }, batched_indexing: None }
Distributing and pruning postings: 5 secs
Number of posting lists: 67756
Building summaries: 145 secs


In [6]:
import numpy as np
import json

file_path = ""

queries = []
with open(file_path, 'r') as f:
    for line in f:
        queries.append(json.loads(line))

MAX_TOKEN_LEN = 30
string_type  = f'U{MAX_TOKEN_LEN}'

queries_ids = np.array([q['id'] for q in queries], dtype=string_type)

query_components = []
query_values = []

for query in queries[:1000]:
    vector = query['vector']
    query_components.append(np.array(list(vector.keys()), dtype=string_type))
    query_values.append(np.array(list(vector.values()), dtype=np.float32))

In [7]:
results = index.batch_search(
    queries_ids=queries_ids,
    query_components=query_components,
    query_values=query_values,
    k=10,
    query_cut=20,
    heap_factor=0.7,
    n_knn=0,
    sorted=True, #specified even if default value
    num_threads=1,
)